* project 目錄為 `End-to-End-Machine-Learning-Pipeline`，以下路徑均以此為 根目錄

* `src/mlProject/constants/__init__.py` 下定義了

	```python
	CONFIG_FILE_PATH
	PARAMS_FILE_PATH
	SCHEMA_FILE_PATH
	```

* `src/mlProject/constants/__init__.py` 要先完成 (針對 5. Update the configuration manager )

# 自行實作

In [ ]:
!git clone https://github.com/henrykohl/MLOps-Foundation.git

In [5]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [ ]:
!conda create -n mlproj python=3.8 -y

In [2]:
%cd MLOps-Foundation/End-to-End-Machine-Learning-Pipeline/

/content/MLOps-Foundation/End-to-End-Machine-Learning-Pipeline


In [12]:
%pwd

'/content/MLOps-Foundation/End-to-End-Machine-Learning-Pipeline'

In [ ]:
!source activate mlproj; pip install -r requirements.txt

* colab 操作時，依序
  - 將 `01_data_ingestion.ipynb` 中，自行實作的執行部分複製過來，執行一次。
  - 將 `02_data_validation.ipynb` 中，自行實作的執行部分複製過來，執行一次。

In [9]:
%%bash
source activate mlproj

python

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path           ## 實例化時，儲存的類型是 str
    data_path: Path

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,   ## 輸出: PosixPath("config/config.yaml")
        params_filepath = PARAMS_FILE_PATH,   ## 輸出: PosixPath("params.yaml")
        schema_filepath = SCHEMA_FILE_PATH):  ## 輸出: PosixPath("schema.yaml")

        self.config = read_yaml(config_filepath) ## 輸出: ConfigBox({...}); config.artifacts_root 是 str
        self.params = read_yaml(params_filepath) ## 輸出: ConfigBox({...})
        self.schema = read_yaml(schema_filepath) ## 輸出: ConfigBox({...})

        create_directories([self.config.artifacts_root]) ## 建立目錄 artifacts



    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation ## 輸出: ConfigBox({...});

        create_directories([config.root_dir]) ## 建目錄 artifacts/data_ingestion; config.root_dir 是 str

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,         ## config.root_dir 是 str
            data_path=config.data_path,       ## config.data_path 是 str
        )

        return data_transformation_config

import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path) ## 檔案 artifacts/data_ingestion/winequality-red.csv 路徑

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)
				## 'train' 存檔路徑 artifacts/data_transformation/train.csv
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
				## 'test' 存檔路徑 artifacts/data_transformation/test.csv
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)


try:
    config = ConfigurationManager()                                            ## 例化configuration，建立主目錄
    data_transformation_config = config.get_data_transformation_config()       ## 執行configuration，建立次目錄，例化entit
    data_transformation = DataTransformation(config=data_transformation_config)## 例化 component
    data_transformation.train_test_spliting()                                  ## 執行 component，進行資料分割
except Exception as e:
    raise e

[2025-12-16 07:24:01,048: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-12-16 07:24:01,049: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-16 07:24:01,051: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-16 07:24:01,051: INFO: common: created directory at: artifacts]
[2025-12-16 07:24:01,052: INFO: common: created directory at: artifacts/data_transformation]
[2025-12-16 07:24:01,083: INFO: <stdin>: Splited data into training and test sets]
[2025-12-16 07:24:01,083: INFO: <stdin>: (1199, 12)]
[2025-12-16 07:24:01,083: INFO: <stdin>: (400, 12)]
(1199, 12)
(400, 12)


# Lecture Demo

In [ ]:
import os
os.chdir("../")
%pwd

'd:\\Bappy\\Live Sessions\\Euron\\MLOPs Masters Batch\\End-to-End-Machine-Learning-Pipeline'

## 4. Update the entity
* project 對應 `src/mlProject/entity/config_entity.py`

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path           ## 實例化時，儲存的類型是 str
    data_path: Path          ## 實例化時，儲存的類型是 str

## 5. Update the configuration manager
* project 對應 `src/mlProject/config/configuration.py`用到
	- '4. entity'： `src/mlProject/entity/config_entity.py` -- 輸出 DataTransformationConfig

In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,   ## 輸出: PosixPath("config/config.yaml")
        params_filepath = PARAMS_FILE_PATH,   ## 輸出: PosixPath("params.yaml")
        schema_filepath = SCHEMA_FILE_PATH):  ## 輸出: PosixPath("schema.yaml")

        self.config = read_yaml(config_filepath) ## 輸出: ConfigBox({...}); config.artifacts_root 是 str
        self.params = read_yaml(params_filepath) ## 輸出: ConfigBox({...})
        self.schema = read_yaml(schema_filepath) ## 輸出: ConfigBox({...})

        create_directories([self.config.artifacts_root]) ## 建立目錄 artifacts



    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation ## 輸出: ConfigBox({...});

        create_directories([config.root_dir]) ## 建目錄 artifacts/data_ingestion; config.root_dir 是 str

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,         ## config.root_dir 是 str
            data_path=config.data_path,       ## config.data_path 是 str
        )

        return data_transformation_config

## 6. Update the components
* project 對應 `src/mlProject/components/data_transformation.py` 用到
	- '4. entity'： `src/mlProject/entity/config_entity.py` -- 輸入 DataTransformationConfig

In [ ]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path) ## 檔案 artifacts/data_ingestion/winequality-red.csv 路徑

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)
				## 'train' 存檔路徑 artifacts/data_transformation/train.csv
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
				## 'test' 存檔路徑 artifacts/data_transformation/test.csv
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)


## 7. Update the pipeline
* project 對應 `src/mlProject/pipeline/stage_03_data_transformation.py` 需調用
	- '5. configuration': `src/mlProject/config/configuration.py` -- 輸出 DataTransformationConfig
	- '6. components': `src/mlProject/components/data_ingestion.py` -- 輸入 DataTransformationConfig

In [ ]:
try:
    config = ConfigurationManager()                                            ## 例化configuration，建立主目錄
    data_transformation_config = config.get_data_transformation_config()       ## 執行configuration，建立次目錄，例化entit
    data_transformation = DataTransformation(config=data_transformation_config)## 例化 component
    data_transformation.train_test_spliting()                                  ## 執行 component，進行資料分割
except Exception as e:
    raise e

[2025-01-12 12:51:51,319: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-12 12:51:51,320: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-12 12:51:51,321: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-12 12:51:51,322: INFO: common: created directory at: artifacts]
[2025-01-12 12:51:51,323: INFO: common: created directory at: artifacts/data_transformation]
[2025-01-12 12:51:51,335: INFO: 741865018: Splited data into training and test sets]
[2025-01-12 12:51:51,335: INFO: 741865018: (1199, 12)]
[2025-01-12 12:51:51,336: INFO: 741865018: (400, 12)]
(1199, 12)
(400, 12)
